# Joblib

Набор утилит для распараллеливания задач, эффективного дискового кэширования и сериализации

In [1]:
import tempfile

import numpy as np

import joblib
from joblib import Memory, Parallel, delayed

### Кэширование

In [2]:
with tempfile.TemporaryDirectory() as cachedir:
    mem = Memory(cachedir)
    
    @mem.cache
    def long_running_func(arr: np.ndarray) -> np.ndarray:
        return np.linalg.pinv(arr)
    
    
    arr = np.random.random(size=(1000, 1000))
    long_running_func(arr)
    long_running_func(arr)
        

________________________________________________________________________________
[Memory] Calling __main__--home-alex-work-university-courses-ml_course-classification-2021-examples-<ipython-input-8f65b72960ea>.long_running_func...
long_running_func(array([[0.352053, ..., 0.211678],
       ...,
       [0.265521, ..., 0.256596]]))
________________________________________________long_running_func - 1.1s, 0.0min


### Параллельные вычисления

In [3]:
import os
import time
from typing import List

def func(x: np.ndarray) -> np.float:
    time.sleep(1)
    return np.linalg.pinv(x ** 5 + x).sum()

matrixes = [np.random.randn(1000, 1000) for _ in range(8)]

def func_parallel(matrixes: List[np.ndarray]):
    with Parallel(n_jobs=os.cpu_count(), backend="multiprocessing") as parallel:
        result = parallel([delayed(func)(matr) for matr in matrixes])
        return result
        
def func_seq(matrixes: List[np.ndarray]):
    result = [func(matr) for matr in matrixes]
    return result

In [4]:
%time r1 = func_parallel(matrixes)

CPU times: user 59.6 ms, sys: 51.1 ms, total: 111 ms
Wall time: 10.5 s


In [5]:
%time r2 = func_seq(matrixes)

CPU times: user 19 s, sys: 6.71 s, total: 25.7 s
Wall time: 15 s


### Сериализация

In [6]:
with tempfile.NamedTemporaryFile(suffix='.lz4') as fn:
    
    big_array = np.random.random(size=(1000, 1000))
    
    with open(fn.name, 'wb') as f:
        joblib.dump(big_array, f)
        
    print(joblib.load(open(fn.name, 'rb')))

[[0.33925968 0.51173772 0.24037399 ... 0.55335966 0.18091704 0.77346507]
 [0.83479128 0.82199771 0.80003229 ... 0.51364328 0.14649307 0.3624289 ]
 [0.26438163 0.61131125 0.16994692 ... 0.73150441 0.06054313 0.4217644 ]
 ...
 [0.86157206 0.41150795 0.05905909 ... 0.64201443 0.75577104 0.82069727]
 [0.66216295 0.69369048 0.09390891 ... 0.8337039  0.5957274  0.01403108]
 [0.39128545 0.3325238  0.40774211 ... 0.4559801  0.90186941 0.29816189]]
